<a href="https://colab.research.google.com/github/ColeDCrawford/mp-scrape/blob/master/MP_Routes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re

import pprint
pp = pprint.PrettyPrinter(indent=2)

idPattern = re.compile("^[0-9]{1,10}$")
uniqueRouteIds = set()

import pandas as pd
from google.colab import files
import time

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
apikey=''
all_routes = []

import csv
route_location_ids = {}
with open("/content/gdrive/My Drive/Mountain Project Scrape/route_loc_ids.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    route_location_ids[int(row['routeId'])] = int(row['locationId'])

with open("/content/gdrive/My Drive/Mountain Project Scrape/route_loc_ids-after-indiana.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    route_location_ids[int(row['routeId'])] = int(row['locationId'])

In [10]:
def get_routes(ids, pause=5):
  time.sleep(pause)
  str_ids = ','.join(str(x) for x in ids)
  params = {
      'key': apikey,
      'routeIds': str_ids
  }
  r = requests.get('https://www.mountainproject.com/data/get-routes', params=params)
  if(r.json()['success'] == 1):
    routes = r.json()['routes']
    # enhance with location ID
    for route in routes:
      locationId = route_location_ids[route['id']]
      route['locationId'] = locationId
    return routes
  else:
    print("Error")
    print(r.json())
    return False

In [7]:
routeIds = list(route_location_ids.keys())
print(f"Total route IDs: {len(routeIds)}")

def divide_chunks(l, n): 
  for i in range(0, len(l), n):  
    yield l[i:i + n]

finished = set()
with open("/content/gdrive/My Drive/Mountain Project Scrape/route_data_v2.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open("/content/gdrive/My Drive/Mountain Project Scrape/route_data.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open("/content/gdrive/My Drive/Mountain Project Scrape/route_data_v3.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open("/content/gdrive/My Drive/Mountain Project Scrape/route_data_v4.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
with open("/content/gdrive/My Drive/Mountain Project Scrape/route_data_v5.csv") as read_obj:
  csv_dict_reader = csv.DictReader(read_obj)
  for row in csv_dict_reader:
    finished.add(int(row['id']))
print(f"Total finished: {len(finished)}")

toProcess = []
for i in routeIds:
  if(i not in finished):
    toProcess.append(i)

print(f"To process: {len(toProcess)}")

Total route IDs: 228287
Total finished: 216000
To process: 12287


In [11]:
chunked = list(divide_chunks(toProcess, 200))
for num, chunk in enumerate(chunked):
  # print(chunk)
  routes = get_routes(chunk)
  last = routes[-1]['id']
  all_routes.extend(routes)
  if((num % 10 == 0) or (num == len(chunked)-1)):
    # write
    df = pd.DataFrame(all_routes)
    df.to_csv(f"/content/gdrive/My Drive/Mountain Project Scrape/route_data_v6.csv", index=False)
    print(f"Previous runs: {len(finished)} | As of last run: {num*200} | Remaining: {len(toProcess) - (num*200)}")

Error
{'success': 0, 'message': 'Rate Limit Exceeded'}


TypeError: ignored

In [25]:
len(all_routes)

32800

In [44]:
print(apikey)

200157044-e39bcc082e1d854263a47239d609d945
